In [3]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Dictionary of class labels
classes = {
    0: "person",
    1: "bicycle",
    2: "car",
    3: "motorcycle",
    4: "bus",
    5: "truck",
    6: "traffic_light",
    7: "stop_sign"
}

In [4]:
def Check_images_no_bounding_boxes(image_roots, label_roots):
    
    image_root = Path(image_roots)
    label_root = Path(label_roots)
    image_dirs = sorted([d for d in image_root.iterdir() if d.is_dir()])
    label_dirs = sorted([d for d in label_root.iterdir() if d.is_dir()])
    
    # Build full list of image-label pairs across all file folders
    # data_pairs = []
    
    for i,j in zip(image_dirs, label_dirs):
        img_folder = i/"ring_front_center"
        lbl_folder = j/ "ring_front_center"
        #data_pairs.append(img_folder)
        images = [d for d in img_folder.iterdir()]
        labels = [d for d in lbl_folder.iterdir()]   
        if len(images) != len(labels):
            ima = [ i.stem.split("_")[-1] for i in images]
            lbl = [ i.stem.split("_")[-1] for i in labels]
            difference = [(i,x) for i,x in enumerate(ima) if x not in lbl]
            for i,_ in difference:
                image_path = images[i]
                
                # Load image using OpenCV
                img = cv2.imread(image_path)
                
                if img is None:
                    print(f"Error loading image: {image_path}")
                    continue
            
                # Convert BGR to RGB (since OpenCV loads images in BGR format)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
                # Display the image using Matplotlib
                plt.figure(figsize=(20, 10))
                plt.imshow(img_rgb)
                plt.axis("off")  # Hide axis
                plt.title(f"Image: {image_path}")
                plt.show()

In [5]:
def image_nbx (image_roots, label_roots):
    
    image_root = Path(image_roots)
    label_root = Path(label_roots)
    image_dirs = sorted([d for d in image_root.iterdir() if d.is_dir()])
    label_dirs = sorted([d for d in label_root.iterdir() if d.is_dir()])
    
    # Build full list of image-label pairs across all file folders
    data_pairs = []
    
    for i,j in zip(image_dirs, label_dirs):
        img_folder = i/"ring_front_center"
        lbl_folder = j/ "ring_front_center"
        #data_pairs.append(img_folder)
        images = [d for d in img_folder.iterdir()]
        labels = [d for d in lbl_folder.iterdir()]   
        if len(images) != len(labels):
            data_pairs.append((img_folder,lbl_folder))
    return data_pairs

In [6]:
# Train paths
image_train = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/images/train"
label_train = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/labels/train"

# Val paths
image_val = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/images/val"
label_val = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/labels/val"


In [14]:
Check_images_no_bounding_boxes(image_train, label_train)

NameError: name 'Check_images_no_bounding_boxes' is not defined

In [15]:
#Check_images_no_bounding_boxes(image_val, label_val)

In [16]:
image_nbx1 = image_nbx(image_train, label_train)
image_nbx1

NameError: name 'image_nbx' is not defined

In [17]:
img_folder, lbl_folder = image_nbx1[1][0], image_nbx1[1][1]
    
images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {}

data_pairs = []
for i in range(len(images)):  
    img_prev = images[i] if i == 0 else images[i - 1]
    img_curr = images[i]
    img_next = images[i] if i == len(images) - 1 else images[i + 1]

    # Get corresponding label for img_curr (instead of img_next)
    lbl_next = labels.get(img_next.stem, "EMPTY_BOUNDING_BOX")  

    data_pairs.append(str(lbl_next))

NameError: name 'image_nbx1' is not defined

In [18]:
len(images), len(labels)

NameError: name 'images' is not defined

In [19]:
data_pairs[-2:]

NameError: name 'data_pairs' is not defined

In [20]:
empty_label_pairs = [pair for pair in data_pairs if pair[2] == "EMPTY_BOUNDING_BOX"]

print(f"Total empty label pairs: {len(empty_label_pairs)}")

NameError: name 'data_pairs' is not defined

In [7]:
import os
from pathlib import Path

def dataset(image_path, label_path):
    image_root = Path(image_path)
    label_root = Path(label_path)
    
    # Get sorted list of subdirectories
    image_dirs = sorted([d for d in image_root.iterdir() if d.is_dir()])
    label_dirs = sorted([d for d in label_root.iterdir() if d.is_dir()])
    
    data_pairs = []
    
    for img_dir, lbl_dir in zip(image_dirs, label_dirs):
        img_folder = img_dir / "ring_front_center"
        lbl_folder = lbl_dir / "ring_front_center"
        
        if not img_folder.exists():
            continue  # Skip if images folder doesn't exist
    
        # Get sorted list of image and label files
        images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
        labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {}
        
        for i in range(len(images)):  
            img_prev = images[i] if i == 0 else images[i - 1]
            img_curr = images[i]
            img_next = images[i] if i == len(images) - 1 else images[i + 1]
            
            # Get corresponding label for img_curr (instead of img_next)
            lbl_next = labels.get(img_next.stem, "EMPTY_BOUNDING_BOX")  
        
            data_pairs.append((str(img_prev), str(img_curr), str(lbl_next)))

    return data_pairs



In [8]:
train_data = dataset(image_train, label_train)
val_data = dataset(image_val, label_val)

len(train_data), len(val_data)

(39384, 15062)

In [9]:
#train_data

In [10]:
def load_label( label_path):
        """Loads bounding box labels from a text file."""
        boxes = []
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = list(map(float, line.strip().split()))
                boxes.append(values)  # Each row: [class, x, y, w, h]
        
        return torch.tensor(boxes, dtype=torch.float32)

data = []

for img_prev_path,img_curr_path, lbl_curr_path in train_data:
    img_prev = cv2.imread(str(img_prev_path))  #  BGR format
    img_curr = cv2.imread(str(img_curr_path))
    
    # Convert BGR to RGB
    img_prev = cv2.cvtColor(img_prev, cv2.COLOR_BGR2RGB)
    img_curr = cv2.cvtColor(img_curr, cv2.COLOR_BGR2RGB)
    
    # Convert numpy array to torch tensor (HWC → CHW)
    img_prev = torch.from_numpy(img_prev).float().permute(2, 0, 1) / 255.0  # Normalize to [0,1]
    img_curr = torch.from_numpy(img_curr).float().permute(2, 0, 1) / 255.0  # Normalize
    
    # Load label or assign empty bounding box
    if lbl_curr_path == "EMPTY_BOUNDING_BOX":
        label = torch.zeros((1, 5))  # Empty label with shape (1, 5) [class, x, y, w, h]
    else:
        label = load_label(lbl_curr_path)
    
    data.append((img_curr, label))

    

NameError: name 'torch' is not defined

In [ ]:
data

In [7]:
empty_label_train = [pair for pair in train_data if pair[2] == "EMPTY_BOUNDING_BOX"]
empty_label_val = [pair for pair in val_data if pair[2] == "EMPTY_BOUNDING_BOX"]
print(f"Total empty label pairs for train: {len(empty_label_train)} and val : {len(empty_label_val)}")

Total empty label pairs for train: 12 and val : 7


In [8]:
image_root = Path(image_val)
label_root = Path(label_val)
image_dirs = sorted([d for d in image_root.iterdir() if d.is_dir()])
label_dirs = sorted([d for d in label_root.iterdir() if d.is_dir()])

# Build full list of image-label pairs across all file folders
data = []

for i,j in zip(image_dirs, label_dirs):
    img_folder = i/"ring_front_center"
    lbl_folder = j/ "ring_front_center"
    # Get sorted list of image and label files
    images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
    labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {} 
    if len(images)!= len(labels):
        print(len(images),len(labels))

889 882


In [1]:
import torch
from torch.utils.data import Dataset
from pathlib import Path
import cv2
import numpy as np

# Train paths
image_train = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/images/train"
label_train = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/labels/train"

# Val paths
image_val = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/images/val"
label_val = r"/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/labels/val"

class AgroverseDataset(Dataset):
    def __init__(self, image_path, label_path, transform=None):
        
        self.image_root = Path(image_path)
        self.label_root = Path(label_path)
        self.transform = transform  # Optional image transformations

        # Get sorted list of subdirectories
        self.image_dirs = sorted([d for d in self.image_root.iterdir() if d.is_dir()])
        self.label_dirs = sorted([d for d in self.label_root.iterdir() if d.is_dir()])

        # Build list of (prev_img, curr_img, label) pairs
        self.data_pairs = self.create_data_pairs()

    def create_data_pairs(self):
        """Builds (prev, curr, label) pairs."""
        data_pairs = []
        
        for img_dir, lbl_dir in zip(self.image_dirs, self.label_dirs):
            img_folder = img_dir / "ring_front_center"
            lbl_folder = lbl_dir / "ring_front_center"

            if not img_folder.exists():
                continue  # Skip if images folder doesn't exist
            
            # Get sorted list of image and label files
            images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
            labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {}
            for i in range(len(images)):  
                img_prev = images[i] if i == 0 else images[i - 1]
                img_curr = images[i]
                img_next = images[i] if i == len(images) - 1 else images[i + 1]
                
                # Get corresponding label for img_curr (instead of img_next)
                lbl_next = labels.get(img_next.stem, "EMPTY_BOUNDING_BOX")  
                data_pairs.append((str(img_prev), str(img_curr), str(lbl_next)))

        return data_pairs

    def __len__(self):
        """Returns the number of data pairs."""
        return len(self.data_pairs)

    def __getitem__(self, idx):
        """Loads an image pair and its corresponding label."""
        img_prev_path, img_curr_path, lbl_curr_path = self.data_pairs[idx]

        # Load images
        img_prev = cv2.imread(str(img_prev_path))  # BGR format
        img_curr = cv2.imread(str(img_curr_path))

        # Convert BGR to RGB
        img_prev = cv2.cvtColor(img_prev, cv2.COLOR_BGR2RGB)
        img_curr = cv2.cvtColor(img_curr, cv2.COLOR_BGR2RGB)

        # Convert numpy array to torch tensor (HWC → CHW)
        img_prev = torch.from_numpy(img_prev).float().permute(2, 0, 1) / 255.0  # Normalize to [0,1]
        img_curr = torch.from_numpy(img_curr).float().permute(2, 0, 1) / 255.0  # Normalize

        # Apply transformations if provided
        if self.transform:
            img_prev = self.transform(img_prev)
            img_curr = self.transform(img_curr)

        # Load label or assign empty bounding box
        if lbl_curr_path == "EMPTY_BOUNDING_BOX":
            label = torch.zeros((1, 5))  # Empty label with shape (1, 5) [class, x, y, w, h]
        else:
            label = self.load_label(lbl_curr_path)

        return img_prev, img_curr, label

    def load_label(self, label_path):
        """Loads bounding box labels from a text file."""
        boxes = []
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = list(map(float, line.strip().split()))
                boxes.append(values)  # Each row: [class, x, y, w, h]
        
        return torch.tensor(boxes, dtype=torch.float32)

# Create dataset
dataset = AgroverseDataset(
    image_path=image_train, 
    label_path=label_train, 
    image_size= 640
)
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    imgs_prev, imgs_curr, labels = zip(*batch)  # Unpack batch

    imgs_prev = torch.stack(imgs_prev)  # Stack images into a tensor
    imgs_curr = torch.stack(imgs_curr)

    # Pad labels to the max number of bounding boxes in the batch
    labels = [torch.tensor(lbl) for lbl in labels]  # Convert lists to tensors
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)  # Padding value -1

    return imgs_prev, imgs_curr, labels_padded



dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Iterate over data

for img_prev, img_curr, labels in dataloader:
    print(img_prev.shape, img_curr.shape, labels.shape)  # Example output
    break
    
    

TypeError: AgroverseDataset.__init__() got an unexpected keyword argument 'image_size'

In [ ]:
print(i)

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    imgs_prev, imgs_curr, labels = zip(*batch)  # Unpack batch

    imgs_prev = torch.stack(imgs_prev)  # Stack images into a tensor
    imgs_curr = torch.stack(imgs_curr)

    # Pad labels to the max number of bounding boxes in the batch
    labels = [torch.tensor(lbl) for lbl in labels]  # Convert lists to tensors
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)  # Padding value -1

    return imgs_prev, imgs_curr, labels_padded



dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Iterate over data
for img_prev, img_curr, labels in dataloader:
    print(img_prev.shape, img_curr.shape, labels.shape)  # Example output
    
    

In [164]:

nvidia-smi

NameError: name 'nvidia' is not defined

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import cv2
import numpy as np
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

# Define target image size
IMAGE_SIZE = (640, 640)  # (height, width)

class AgroverseDataset(Dataset):
    def __init__(self, image_path, label_path, transform=None):
        self.image_root = Path(image_path)
        self.label_root = Path(label_path)
        self.transform = transform  # Optional image transformations

        # Get sorted list of subdirectories
        self.image_dirs = sorted([d for d in self.image_root.iterdir() if d.is_dir()])
        self.label_dirs = sorted([d for d in self.label_root.iterdir() if d.is_dir()])

        # Build list of (prev_img, curr_img, label) pairs
        self.data_pairs = self.create_data_pairs()

    def create_data_pairs(self):
        """Builds (prev, curr, label) pairs."""
        data_pairs = []
        
        for img_dir, lbl_dir in zip(self.image_dirs, self.label_dirs):
            img_folder = img_dir / "ring_front_center"
            lbl_folder = lbl_dir / "ring_front_center"

            if not img_folder.exists():
                continue  # Skip if images folder doesn't exist
            
            # Get sorted list of image and label files
            images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
            labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {}

            for i in range(len(images)):  
                img_prev = images[i] if i == 0 else images[i - 1]
                img_curr = images[i]
                
                img_next = images[i] if i == len(images) - 1 else images[i + 1]
                
                # Get corresponding label for img_curr (instead of img_next)
                lbl_next = labels.get(img_next.stem, "EMPTY_BOUNDING_BOX")  
                data_pairs.append((str(img_prev), str(img_curr), str(lbl_next)))

        return data_pairs

    def __len__(self):
        """Returns the number of data pairs."""
        return len(self.data_pairs)

    def __getitem__(self, idx):
        """Loads an image pair and its corresponding label."""
        img_prev_path, img_curr_path, lbl_curr_path = self.data_pairs[idx]

        # Load images
        img_prev = cv2.imread(str(img_prev_path))  # BGR format
        img_curr = cv2.imread(str(img_curr_path))

        # Convert BGR to RGB
        img_prev = cv2.cvtColor(img_prev, cv2.COLOR_BGR2RGB)
        img_curr = cv2.cvtColor(img_curr, cv2.COLOR_BGR2RGB)

        # Resize images & adjust bounding boxes
        img_prev, _ = self.resize_image_with_bbox(img_prev, None)
        img_curr, label = self.resize_image_with_bbox(img_curr, lbl_curr_path)

        # Convert numpy array to torch tensor (HWC → CHW)
        img_prev = torch.from_numpy(img_prev).float().permute(2, 0, 1) / 255.0  # Normalize
        img_curr = torch.from_numpy(img_curr).float().permute(2, 0, 1) / 255.0  # Normalize

        return img_prev, img_curr, label

    def resize_image_with_bbox(self, image, label_path):
        """Resize image while keeping aspect ratio and adjust bounding boxes."""
        h, w, _ = image.shape
        new_w, new_h = IMAGE_SIZE

        # Compute scaling factor while keeping aspect ratio
        scale = min(new_w / w, new_h / h)
        resized_w, resized_h = int(w * scale), int(h * scale)
        image_resized = cv2.resize(image, (resized_w, resized_h))

        # Create a blank canvas (padded image)
        image_padded = np.zeros((new_h, new_w, 3), dtype=np.uint8)
        pad_x = (new_w - resized_w) // 2
        pad_y = (new_h - resized_h) // 2

        # Paste resized image onto padded canvas
        image_padded[pad_y:pad_y + resized_h, pad_x:pad_x + resized_w, :] = image_resized

        # If no labels exist, return empty tensor
        if label_path == "EMPTY_BOUNDING_BOX":
            return image_padded, torch.zeros((1, 5))  # Empty label with shape (1,5)

        # Load bounding boxes
        bboxes = self.load_label(label_path)

        # Adjust bounding boxes
        new_bboxes = []
        for bbox in bboxes:
            class_id, x, y, w, h = bbox  # YOLO format: [class, x_center, y_center, width, height]

            # Scale bounding box coordinates
            x = x * scale + pad_x
            y = y * scale + pad_y
            w = w * scale
            h = h * scale

            new_bboxes.append([class_id, x, y, w, h])

        return image_padded, torch.tensor(new_bboxes, dtype=torch.float32)

    def load_label(self, label_path):
        """Loads bounding box labels from a text file."""
        boxes = []
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = list(map(float, line.strip().split()))
                boxes.append(values)  # Each row: [class, x_center, y_center, width, height]
        
        return torch.tensor(boxes, dtype=torch.float32)

# Create dataset
dataset = AgroverseDataset(
    image_path=image_train, 
    label_path=label_train
)

# Define Collate Function for Padding Labels
def collate_fn(batch):
    imgs_prev, imgs_curr, labels = zip(*batch)  # Unpack batch

    imgs_prev = torch.stack(imgs_prev)  # Stack images into a tensor
    imgs_curr = torch.stack(imgs_curr)

    # Pad labels to the max number of bounding boxes in the batch
    labels = [torch.tensor(lbl) for lbl in labels]  # Convert lists to tensors
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)  # Padding value -1

    return imgs_prev, imgs_curr, labels_padded

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Iterate over data
for img_prev, img_curr, labels in dataloader:
    print(img_prev.shape, img_curr.shape, labels.shape)  # Expected: (32, 3, 640, 640)
    break


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.nn.utils.rnn import pad_sequence

# Define target image size
IMAGE_SIZE = (600, 640)  # (height, width)

# Define augmentation pipeline (Resize + Normalize + Convert to Tensor)
transform = A.Compose([
    A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),  # Resize images and bounding boxes
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalize like ImageNet
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Dataset Paths
image_train = "/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/images/train"
label_train = "/home/nihar22951/Online_detection/Online_detection/Datasets/Agroverse/Argoverse-1.1/labels/train"

class AgroverseDataset(Dataset):
    def __init__(self, image_path, label_path, transform=None):
        self.image_root = Path(image_path)
        self.label_root = Path(label_path)
        self.transform = transform  # Optional image transformations

        # Get sorted list of subdirectories
        self.image_dirs = sorted([d for d in self.image_root.iterdir() if d.is_dir()])
        self.label_dirs = sorted([d for d in self.label_root.iterdir() if d.is_dir()])

        # Build list of (prev_img, curr_img, label) pairs
        self.data_pairs = self.create_data_pairs()

    def create_data_pairs(self):
        """Builds (prev, curr, label) pairs."""
        data_pairs = []
        
        for img_dir, lbl_dir in zip(self.image_dirs, self.label_dirs):
            img_folder = img_dir / "ring_front_center"
            lbl_folder = lbl_dir / "ring_front_center"

            if not img_folder.exists():
                continue  # Skip if images folder doesn't exist
            
            # Get sorted list of image and label files
            images = sorted([f for f in img_folder.iterdir() if f.suffix == ".jpg"])
            labels = {f.stem: f for f in lbl_folder.iterdir() if f.suffix == ".txt"} if lbl_folder.exists() else {}

            for i in range(len(images)):  
                img_prev = images[i] if i == 0 else images[i - 1]
                img_curr = images[i]
                img_next = images[i] if i == len(images) - 1 else images[i + 1]
                
                # Get corresponding label for img_curr (instead of img_next)
                lbl_next = labels.get(img_next.stem, "EMPTY_BOUNDING_BOX")  
                data_pairs.append((str(img_prev), str(img_curr), str(lbl_next)))

        return data_pairs

    def __len__(self):
        """Returns the number of data pairs."""
        return len(self.data_pairs)

    def __getitem__(self, idx):
        """Loads an image pair and its corresponding label."""
        img_prev_path, img_curr_path, lbl_curr_path = self.data_pairs[idx]

        # Load images
        img_prev = cv2.imread(img_prev_path)  # BGR format
        img_curr = cv2.imread(img_curr_path)

        # Convert BGR to RGB
        img_prev = cv2.cvtColor(img_prev, cv2.COLOR_BGR2RGB)
        img_curr = cv2.cvtColor(img_curr, cv2.COLOR_BGR2RGB)

        # Load bounding boxes
        if lbl_curr_path == "EMPTY_BOUNDING_BOX":
            bboxes = []
            class_labels = []
        else:
            bboxes, class_labels = self.load_label(lbl_curr_path)

        # Apply Albumentations transform (resize images + scale bounding boxes)
        transformed = self.transform(image=img_prev, bboxes=bboxes, class_labels=class_labels)
        img_prev = transformed["image"]
        transformed = self.transform(image=img_curr, bboxes=bboxes, class_labels=class_labels)
        img_curr = transformed["image"]

        # Convert bounding boxes to tensor
        if bboxes:
            label_tensor = torch.tensor([class_labels] + bboxes, dtype=torch.float32)
        else:
            label_tensor = torch.zeros((1, 5))  # Empty label

        return img_prev, img_curr, label_tensor

    def load_label(self, label_path):
        """Loads bounding box labels from a text file."""
        bboxes = []
        class_labels = []
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = list(map(float, line.strip().split()))
                class_labels.append(values[0])  # Class label
                bboxes.append(values[1:])  # x, y, w, h (Pascal VOC format)
        
        return bboxes, class_labels

# Create dataset
dataset = AgroverseDataset(
    image_path=image_train, 
    label_path=label_train, 
    transform=transform
)

# Define Collate Function for Padding Labels
def collate_fn(batch):
    imgs_prev, imgs_curr, labels = zip(*batch)  # Unpack batch

    imgs_prev = torch.stack(imgs_prev)  # Stack images into a tensor
    imgs_curr = torch.stack(imgs_curr)

    # Pad labels to the max number of bounding boxes in the batch
    labels = [torch.tensor(lbl) for lbl in labels]  # Convert lists to tensors
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)  # Padding value -1

    return imgs_prev, imgs_curr, labels_padded

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Iterate over data
for img_prev, img_curr, labels in dataloader:
    print(img_prev.shape, img_curr.shape, labels.shape)  # Expected: (32, 3, 640, 640)
    break


ValueError: x_max is less than or equal to x_min for bbox [2.5865342e-04 4.8576389e-04 5.7237412e-05 1.0625000e-04 2.0000000e+00].

In [1]:
from torch.utils.data import DataLoader

#dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=4)


In [2]:
from torch.utils.data.distributed import DistributedSampler

# sampler = DistributedSampler(dataset, shuffle=True)

# dataloader = DataLoader(dataset, batch_size=32, sampler=sampler, collate_fn=collate_fn, num_workers=4)


In [1]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Example bounding box sequences (each tensor is an image's bboxes)
bboxes = [
    torch.tensor([[0, 0.5, 0.5, 0.2, 0.3],  # Image 1 (3 objects)
                  [1, 0.2, 0.3, 0.1, 0.1],
                  [2, 0.7, 0.8, 0.2, 0.2]]),
    
    torch.tensor([[0, 0.4, 0.5, 0.2, 0.2],  # Image 2 (5 objects)
                  [1, 0.3, 0.2, 0.15, 0.1],
                  [1, 0.8, 0.9, 0.25, 0.25],
                  [2, 0.6, 0.6, 0.2, 0.2],
                  [0, 0.1, 0.1, 0.1, 0.1]])
]

# Pad sequences to match the max number of bboxes (5 in this case)
padded_bboxes = pad_sequence(bboxes, batch_first=True, padding_value=-1)

print(padded_bboxes)


tensor([[[ 0.0000,  0.5000,  0.5000,  0.2000,  0.3000],
         [ 1.0000,  0.2000,  0.3000,  0.1000,  0.1000],
         [ 2.0000,  0.7000,  0.8000,  0.2000,  0.2000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000]],

        [[ 0.0000,  0.4000,  0.5000,  0.2000,  0.2000],
         [ 1.0000,  0.3000,  0.2000,  0.1500,  0.1000],
         [ 1.0000,  0.8000,  0.9000,  0.2500,  0.2500],
         [ 2.0000,  0.6000,  0.6000,  0.2000,  0.2000],
         [ 0.0000,  0.1000,  0.1000,  0.1000,  0.1000]]])
